In [ ]:
# !sudo apt-get install tesseract-ocr-* -y
# 

In [6]:
import cv2
import numpy as np
import pdf2image
import pytesseract

# Extract page 3 from PDF in proper quality
page_3 = np.array(pdf2image.convert_from_path('KUPRIN KI KAHANIYAN - HINDI.pdf',
                                              first_page=7, last_page=8,
                                              dpi=300, grayscale=True)[0])

# Inverse binarize for contour finding
thr = cv2.threshold(page_3, 128, 255, cv2.THRESH_BINARY_INV)[1]

# Find contours w.r.t. the OpenCV version
cnts = cv2.findContours(thr, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

# STEP 1: Extract texts outside of the two tables

# Mask out the two tables
cnts_tables = [cnt for cnt in cnts if cv2.contourArea(cnt) > 10000]
no_tables = cv2.drawContours(thr.copy(), cnts_tables, -1, 0, cv2.FILLED)

# Find bounding rectangles of texts outside of the two tables
no_tables = cv2.morphologyEx(no_tables, cv2.MORPH_CLOSE, np.full((21, 51), 255))
cnts = cv2.findContours(no_tables, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
rects = sorted([cv2.boundingRect(cnt) for cnt in cnts], key=lambda r: r[1])

# Extract texts from each bounding rectangle
print('\nExtract texts outside of the two tables\n')
for (x, y, w, h) in rects:
    text = pytesseract.image_to_string(page_3[y:y+h, x:x+w],
                                       config='--psm 6', lang='tam')
    text = text.replace('\n', '').replace('\f', '')
    print('x: {}, y: {}, text: {}'.format(x, y, text))

# STEP 2: Extract texts from inside of the two tables

rects = sorted([cv2.boundingRect(cnt) for cnt in cnts_tables],
               key=lambda r: r[1])

# Iterate each table
for i_r, (x, y, w, h) in enumerate(rects, start=1):

    # Find bounding rectangles of cells inside of the current table
    cnts = cv2.findContours(page_3[y+2:y+h-2, x+2:x+w-2],
                            cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    inner_rects = sorted([cv2.boundingRect(cnt) for cnt in cnts],
                         key=lambda r: (r[1], r[0]))

    # Extract texts from each cell of the current table
    print('\nExtract texts inside table {}\n'.format(i_r))
    for (xx, yy, ww, hh) in inner_rects:

        # Set current coordinates w.r.t. full image
        xx += x
        yy += y

        # Get current cell
        cell = page_3[yy+2:yy+hh-2, xx+2:xx+ww-2]

        # For table 1, simply extract texts as-is
        if i_r == 1:
            text = pytesseract.image_to_string(cell, config='--psm 6',
                                               lang='tam')
            text = text.replace('\n', '').replace('\f', '')
            print('x: {}, y: {}, text: {}'.format(xx, yy, text))

        # For table 2, extract single elements
        if i_r == 2:

            # Floodfill rectangles around numbers
            ys, xs = np.min(np.argwhere(cell == 0), axis=0)
            temp = cv2.floodFill(cell.copy(), None, (xs, ys), 255)[1]
            mask = cv2.floodFill(thr[yy+2:yy+hh-2, xx+2:xx+ww-2].copy(),
                                 None, (xs, ys), 0)[1]

            # Extract left (Hindi) and right (English) parts
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE,
                                    np.full((2 * hh, 5), 255))
            cnts = cv2.findContours(mask,
                                    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            boxes = sorted([cv2.boundingRect(cnt) for cnt in cnts],
                           key=lambda b: b[0])

            # Extract texts from each part of the current cell
            for i_b, (x_b, y_b, w_b, h_b) in enumerate(boxes, start=1):

                # For the left (Hindi) part, extract Hindi texts
                if i_b == 1:

                    text = pytesseract.image_to_string(
                        temp[y_b:y_b+h_b, x_b:x_b+w_b],
                        config='--psm 6',
                        lang='tam')
                    text = text.replace('\f', '')

                # For the left (English) part, extract English texts
                if i_b == 2:

                    text = pytesseract.image_to_string(
                        temp[y_b:y_b+h_b, x_b:x_b+w_b],
                        config='--psm 6',
                        lang='eng')
                    text = text.replace('\f', '')

                print('x: {}, y: {}, text:\n{}'.format(xx, yy, text))


Extract texts outside of the two tables

x: 234, y: 220, text: ன்‌ ஸீ 2ளின்தி சோம்ளிஎன்‌ மூளிஎ்‌ எனி & ஏகல்‌ 895 ௭₹ இன்‌சளி எளி எள்‌ ளி டன்சன்‌& எனா ஏடு ளி: ௩௭ ளிஎ சாக எர்ண்தேள்‌ எர எனி எண்சர்‌ ரர ரி எ எனிவிண ணாக ள்கி எ எிணாளா ட னாள்‌ னி எ ளஎ(எணரா ஏணின்‌ என்‌ ரா எள்ளி சா எளி ஈ4 என்னஎரர்‌ என்‌ 4 | எரின உ ண்ள்‌ எ ஏடு எண்‌ எச எகா ள்‌ஸ்கா ஏன்‌ ராக சானி ளி ட ஈனிஎ கா எர்சு, ஜா ரளி்ள ரா நீரி ர்௭வி அனினின்‌, ஈகிள்‌ ஜாரா ஏகம்‌ ஜி கரி ஏட 4 ௭எள சரச கேஜ சொ 8 ரச்‌ ஏ எளி 848) எளி எஏள் கா கின ரன (ர ஈசா ள்‌ |- எாண்ள்என்ஏ என்ர எளி எனன ஈன்‌ ௭௩ ௭௩8 ௭௭ ௧௭சானா | எண கசா ஜீ எளிது எாற்்ள, ஸ்‌ ரா6%சாரண ளா சோன்‌ என்து எ ஜனாசா எ எரி என்‌என்னி, சான்விச்‌ களா எனா ரளி ரி ஈசா கா எனி-சண எள்‌ காள | கா ௭௭ ஈர ஏக சவளி 86 என்கசம்ஸ்‌ னாகி எ சாக்‌ எர சரசஏ எனா எசா ஏர | ௭௫என்னி ஸிளா எண எளி என்‌ சன எர்கி சாண, சட்‌சாஜி௩ ஊரக எ ள்ள ஊர்‌ உ சாகர 8ஈ% லி எரஜாஸ்‌, சகன்‌ ள்‌ என்‌ கா எ ளா எளிள்ள்‌ ணன்‌ எறிஸ்ள்‌ எக எள்‌ டர எர ௭௭8ளி்ச னி டி ஏரா எர எரி ளி என்‌ எடம்‌ எச ளர் ளி எண ஸ்‌ளா ௭௨ எக எர எனா எ ளள எர | இ75 ௭6 எளி என்‌, எர ஜி: க 3 ஏர 

In [11]:
import cv2
import numpy as np
import pdf2image
import pytesseract

# Extract page 3 from PDF in proper quality
page_3 = np.array(pdf2image.convert_from_path('bengali.pdf',
                                              first_page=8, last_page=10,
                                              dpi=300, grayscale=True)[0])

# Inverse binarize for contour finding
thr = cv2.threshold(page_3, 128, 255, cv2.THRESH_BINARY_INV)[1]

# Find contours w.r.t. the OpenCV version
cnts = cv2.findContours(thr, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

# STEP 1: Extract texts outside of the two tables

# Mask out the two tables
cnts_tables = [cnt for cnt in cnts if cv2.contourArea(cnt) > 10000]
no_tables = cv2.drawContours(thr.copy(), cnts_tables, -1, 0, cv2.FILLED)

# Find bounding rectangles of texts outside of the two tables
no_tables = cv2.morphologyEx(no_tables, cv2.MORPH_CLOSE, np.full((21, 51), 255))
cnts = cv2.findContours(no_tables, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
rects = sorted([cv2.boundingRect(cnt) for cnt in cnts], key=lambda r: r[1])

# Extract texts from each bounding rectangle
print('\nExtract texts outside of the two tables\n')
for (x, y, w, h) in rects:
    text = pytesseract.image_to_string(page_3[y:y+h, x:x+w],
                                       config='--psm 6', lang='ben')
    text = text.replace('\n', '').replace('\f', '')
    print('x: {}, y: {}, text: {}'.format(x, y, text))

# STEP 2: Extract texts from inside of the two tables

rects = sorted([cv2.boundingRect(cnt) for cnt in cnts_tables],
               key=lambda r: r[1])

# Iterate each table
for i_r, (x, y, w, h) in enumerate(rects, start=1):

    # Find bounding rectangles of cells inside of the current table
    cnts = cv2.findContours(page_3[y+2:y+h-2, x+2:x+w-2],
                            cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    inner_rects = sorted([cv2.boundingRect(cnt) for cnt in cnts],
                         key=lambda r: (r[1], r[0]))

    # Extract texts from each cell of the current table
    print('\nExtract texts inside table {}\n'.format(i_r))
    for (xx, yy, ww, hh) in inner_rects:

        # Set current coordinates w.r.t. full image
        xx += x
        yy += y

        # Get current cell
        cell = page_3[yy+2:yy+hh-2, xx+2:xx+ww-2]

        # For table 1, simply extract texts as-is
        if i_r == 1:
            text = pytesseract.image_to_string(cell, config='--psm 6',
                                               lang='ben')
            text = text.replace('\n', '').replace('\f', '')
            print('x: {}, y: {}, text: {}'.format(xx, yy, text))

        # For table 2, extract single elements
        if i_r == 2:

            # Floodfill rectangles around numbers
            ys, xs = np.min(np.argwhere(cell == 0), axis=0)
            temp = cv2.floodFill(cell.copy(), None, (xs, ys), 255)[1]
            mask = cv2.floodFill(thr[yy+2:yy+hh-2, xx+2:xx+ww-2].copy(),
                                 None, (xs, ys), 0)[1]

            # Extract left (Hindi) and right (English) parts
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE,
                                    np.full((2 * hh, 5), 255))
            cnts = cv2.findContours(mask,
                                    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            boxes = sorted([cv2.boundingRect(cnt) for cnt in cnts],
                           key=lambda b: b[0])

            # Extract texts from each part of the current cell
            for i_b, (x_b, y_b, w_b, h_b) in enumerate(boxes, start=1):

                # For the left (Hindi) part, extract Hindi texts
                if i_b == 1:

                    text = pytesseract.image_to_string(
                        temp[y_b:y_b+h_b, x_b:x_b+w_b],
                        config='--psm 6',
                        lang='tam')
                    text = text.replace('\f', '')

                # For the left (English) part, extract English texts
                if i_b == 2:

                    text = pytesseract.image_to_string(
                        temp[y_b:y_b+h_b, x_b:x_b+w_b],
                        config='--psm 6',
                        lang='ben')
                    text = text.replace('\f', '')

                print('x: {}, y: {}, text:\n{}'.format(xx, yy, text))


Extract texts outside of the two tables

x: 123, y: 357, text: 
x: 134, y: 437, text: 
x: 124, y: 463, text: যাংল। সাহিত্যে পরশুরামের আবির্ভাব একেবারে অত্যাশ্চর্য। বিজ্ঞানবিদ রাজশেখর
x: 131, y: 527, text: বন্ছ-_আচার্য প্রুল্লচন্ত্রের উত্তরমাধক, বেঙ্গল কেঙ্গিক্যালের কর্ণধার-তিনি যে "শ্রীস্রী-সিদ্ধেশ্বরী লিমিটেড" লিখবেন, এ কে জানত । ধনস্তত ১৯২২ সালে যখন ধ্ী গ্রথম গল্পটি
x: 112, y: 652, text: ' “পরশুরাম এই ছগ্সনামে প্রকাশিত হয় সেদিন সে পরশুরামের সঙ্গে বভাবণস্তীর মিতভা ষ
x: 136, y: 713, text: কম'বীর রাজশেখরের কোন সাদৃগ্তঠ কেউ খুজে বার করতে পারেন নি-_-কল্পনীও করেন
x: 135, y: 778, text: নি।.''সেদিনের মে জয়যাত্রা আজও অব্যাহত আছে। নানা ঝড়-বাপটা, দুঃখ-শোক
x: 137, y: 837, text: তাকে সইতে হয়েছে বারবার, তার মধ্যেও তার কৌতুকবোধের উৎস শুকোয় নি, তীক্ষ
x: 136, y: 905, text: ব্যলের ক্ষুরধার তীব্রতা অম্লান আছে। যে গন্সট এখানে ছাপা হল সেটি তার সাতাত্তর
x: 137, y: 964, text: বছর বয়সের রচনা, ১৯৫৬ সালে লিখিত । এভে তার শির অক্ষুত|! খানিকট! বোঝা
x: 138, y: 1025, text: যাবে । কথা-সাহিতোর ক্ষেত্র ছাড়াও 